### Dater performance for TabFact_small

In [1]:
import json
with open('./gloc_end2end_tab_fact_test.json') as f:
        dic = json.load(f)

In [2]:
import collections
import pandas as pd
import numpy as np
acc = 0.
states_list = []
ans_1 = []
for key in dic:
    to_union = collections.defaultdict(float)
    it = dic[key]
    table = it['data_item']['table_text']
    stat = it['data_item']['statement'].split('\n')[0]
    states_list.append(stat)
    # print(table)
    ######### col filed################
    preds = it['generations']

    for pred in preds:
        log_prob_mean = pred[2]
        pred = pred[0]
        try:
            # pred = re.findall(pattern,pred[0])[0][1]
            pred = pred.split(':')[-1]
            # print(pred)
            # input()
        except Exception:
            continue

        if pred.count('True')>=1:
            key = 1
            to_union[key] += np.exp(log_prob_mean)

        elif pred.count('False')>=1:
            key = 0
            to_union[key] += np.exp(log_prob_mean)

    if to_union[0] > to_union[1]:
        pred_answer = 0
    else:
        pred_answer = 1
    gt = it['data_item']['label']
    ans_1.append(pred_answer)
    if gt == pred_answer:
        acc += 1
print('ACC:', acc/len(dic))

ACC: 0.8537549407114624


In [3]:
dater_inst = []
cnt = 0
for key in dic:
    cnt += 1
    tmp = ""
    it = dic[key]
#     print(it)
    sub_evi = it['generations'][0][0]
    stat = it['data_item']['statement'].split('\n')
    if len(stat) > 1:
        stat_add = " ".join(stat[1:])
    tmp += sub_evi
    tmp += stat_add
    dater_inst.append(tmp)

In [5]:
dataset = []
with open('./small_test.jsonl') as f:
    lines = f.readlines()
    for line in lines:
        dic = json.loads(line)
        dataset.append(dic)

In [5]:
### make sure statement and table are linking correctly
dataset_same = []
for stat in states_list:
    for item in dataset:
        if stat == item['statement']:
            dataset_same.append(item)
            break
len(dataset_same)

2024

In [6]:
ground_truth = []
for item in dataset_same:
    ground_truth.append(item['label'])
len(ground_truth)

2024

In [7]:
queries = []
for item in dataset_same:
    queries.append(item['statement'])
len(queries)

2024

In [8]:
df_list = []
for data in dataset_same:
    columns = data['table_text'][0]  
    data_rows = data['table_text'][1:]  
    df = pd.DataFrame(data_rows, columns=columns)
    df_list.append(df)
len(df_list)

2024

### Construct testing prompts

In [40]:
def format_header(input_list):
    return ' | ' + ' | '.join(str(item) for item in input_list) + ' |'

In [41]:
def format_nested_list(nested_list):
    result = ''
    for row in nested_list:
        result += '| ' + ' | '.join(str(item) for item in row) + ' |\n'
    return result.strip()

In [12]:
table = format_nested_list(dataset[0]['table_text'][1:])
print(table)

| 1 | sept 20 | ole miss | loss | 7 | 14 | 0 - 1 |
| 2 | sept 27 | cincinnati | win | 20 | 0 | 1 - 1 |
| 3 | oct 4 | xavier | win | 20 | 7 | 2 - 1 |
| 4 | oct 11 | 9 georgia | win | 26 | 0 | 3 - 1 , 20 |
| 5 | oct 18 | 10 vanderbilt | win | 14 | 0 | 4 - 1 , 14 |
| 6 | oct 25 | michigan state | win | 7 | 6 | 5 - 1 , 13 |
| 7 | nov 1 | 18 alabama | loss | 0 | 13 | 5 - 2 |
| 8 | nov 8 | west virginia | win | 15 | 6 | 6 - 2 |
| 9 | nov 15 | evansville | win | 36 | 0 | 7 - 2 |
| 10 | nov 22 | tennessee | loss | 6 | 13 | 7 - 3 |


In [13]:
header = format_header(dataset[0]['table_text'][0])
print(header)

 | game | date | opponent | result | wildcats points | opponents | record |


In [29]:
def extract_reasoning(input_string):
    final_answer_index = input_string.find('```python')
    content = input_string[:final_answer_index].strip()
    return content

In [30]:
import re

def extract_python_code(input_string):
    # Define the pattern to match Python code
    pattern = r'```python(.*?)```'

    # Use re.findall to find all matches
    matches = re.findall(pattern, input_string, re.DOTALL)

    # Return the matched Python code
    return matches[0]

In [31]:
import subprocess

# Your Python script as a string
python_script = """

distances = [59.51, 62.86, 63.50, 62.58, 60.99, 57.79, 56.98]
average_distance = sum(distances) / len(distances)
print("Final Answer:", average_distance)

"""

def run_string(code_string):
  try:
      # Run the script in a subprocess
      result = subprocess.run(['python', '-c', code_string], capture_output=True, text=True, check=True)

      # Print the captured output
      return(result.stdout.strip())

  except subprocess.CalledProcessError as e:
      return(f"An error occurred: {e}")

res = run_string(python_script)
print(res)

Final Answer: 60.60142857142858


In [110]:
## Prompt template
prompt = "Please determine whether a given statement is true or not, according to the given table regarding [TITLE] \n"
prompt += "The table header is: [HEADER]" + '\n'
prompt += "The table context is: " + "[TABLE]" + "\n"
prompt += "The statement is: " + "[STAT]" + "\n"
note = """
Notes:
- Try to solve the problem step by step and give the reasoning process firstly
- Then, give me the python script
- Use print function to output the final answer (note: do not add any extra context in the print)
- Give me the answer in format "Final Answer: Answer" form (either True or False, without any explanation)
- Directly output the answer after the python script
"""
prompt += note
prompt_script = prompt
prompt_script

'Please determine whether a given statement is true or not, according to the given table regarding [TITLE] \nThe table header is: [HEADER]\nThe table context is: [TABLE]\nThe statement is: [STAT]\n\nNotes:\n- Try to solve the problem step by step and give the reasoning process firstly\n- Then, give me the python script\n- Use print function to output the final answer (note: do not add any extra context in the print)\n- Give me the answer in format "Final Answer: Answer" form (either True or False, without any explanation)\n- Directly output the answer after the python script\n'

In [30]:
## generate_prompt using prompt template to generate each question
def generate_prompt(data: list, prompt_temp: str)->str:
  table_context = format_nested_list(data['table_text'][1:])
  statement = data['statement']
  table_title = data['table_caption']
  table_header = format_header(data['table_text'][0])
  prompt = prompt_temp.replace("[TITLE]", table_title)
  prompt = prompt.replace("[HEADER]", table_header)
  prompt = prompt.replace("[TABLE]", table_context)
  prompt = prompt.replace("[STAT]", statement)
  return prompt

In [132]:
## get prompt for each question
prompt_list = []
for data in dataset_same:
  prompt_i = generate_prompt(data, prompt_script)
  prompt_list.append(prompt_i)

In [12]:
## get final answer
import re
def filter_answer(raw_context) ->str:
  pattern = r'Final Answer: (.*)'
  result = re.findall(pattern, raw_context)
  print(result)
  return result

In [34]:
import requests

def request_gpt4(prompt, api_key, model="gpt-4o-mini", temperature=0):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature
    }

    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

In [80]:
api_key = "YOUR_API_KEY"
# response = request_gpt4(prompt_list[0], api_key)
# print(response)
# resp_context = response['choices'][0]['message']['content']
# print(resp_context)

In [6]:
refine_propose = []
org_propose = []
instruction = []

for i in range(len(prompt_list)):
  print(i)
  filter_ans = ""
  python_res = ""
  response = request_gpt35(prompt_list[i], api_key)
  try:
      respon = response['choices'][0]['message']['content']
  except:
      org_propose.append('')
      instruction.append('')
      refine_propose.append('False')
      continue
  org_propose.append(respon)
  instruct = extract_reasoning(respon)
  instruction.append(instruct)
  if "python" in respon:
    try:
      python_code = extract_python_code(respon)
      python_res = run_string(python_code)
      if "Final Answer" in python_res and "error" not in python_res:
        python_res = filter_answer(python_res)
    except:
      python_res = ""
    respon = respon.replace(python_code, "")
  if "Final Answer" in respon:
      filter_ans = filter_answer(respon)

  if not filter_ans:
    refine_propose.append(python_res)
  elif not python_res:
    refine_propose.append(filter_ans)
  elif filter_ans and python_res:
    if "error" in python_res:
      refine_propose.append(filter_ans)
    else:
        refine_propose.append(filter_ans)
  else:
    refine_propose.append("no result")

### Calculate Accuracy for Numsolver

In [38]:
cnt = 0
num_ans = []
for i in range(len(refine_propose)):
    ans = 0
    try:
        if refine_propose[i][0].strip() == 'True':
            ans = 1
    except:
        ans = -1
    num_ans.append(ans)
    if ans == ground_truth[i]:
        cnt += 1
cnt / len(refine_propose)

0.8898221343873518

In [70]:
ans_refine = []
for i, a in enumerate(ans):
    if type(a) != list:
        if a == "True":
            ans_refine.append(1)
        else:
            ans_refine.append(0)
    else:
        if a == ['True']:
            ans_refine.append(1)
        else:
            ans_refine.append(0)

In [168]:
i = 0
acc = 0.
for key in dic:
    to_union = collections.defaultdict(float)
    it = dic[key]
    # CodeX 没有产生任何东西
    table = it['data_item']['table_text']
    stat = it['data_item']['statement'].split('\n')[0]
    states_list.append(stat)

    pred = ans_refine[i]
    gt = it['data_item']['label']
    if gt == pred:
        acc += 1
    i += 1
print('ACC:', acc/len(dic))

ACC: 0.8799407114624506


## AnswerSelector

In [ ]:
# Load answers from two branches (NumSolver and Dater)
# You need to load your answers here and construct prompts
# Our example prompts are in 
exp_ans_sel_prompts = "TabLaP/results/exp_prompts/tabfact/ans_sel_prompts.txt"
exp_tw_eval_prompts = "TabLaP/results/exp_prompts/tabfact/tw_eval_prompts.txt"
# Our tablap results is in
tab_lap_ans_path = "TabLaP/tabfact/tablap_res.txt"
# You can read with:
tab_lap_ans = []
with open(tab_lap_ans_path, "r", encoding="utf-8") as f:
    file_content = f.read()
    tab_lap_ans = file_content.split("special_split")[:-1]

In [57]:
prompt_cls = """
Below is a table header regarding [TITLE]:
Header: [HEADER]
You're tasked with determing whether the given statement is True:
[STAT]
You have 2 answers with their corresponding reasoning processes derived by two different models. Answer [A] was derived by the A model. Answer [B] was derived by B model.
For the answers, 1 means True and 0 means False
Reasoning of Answer [A] is: [RES1]
Answer [A] is: [ANS1]
Reasoning of Answer [B] is: [RES2]
Answer [B] is: [ANS2]
Your task is to determine which is the correct answer. The final answer is [A] if Answer A is correct, and [B] if Answer B is correct.
If Answer [A] and Answer [B] are the same, the final answer could be either [A] or [B].
Therefore, the final answer is:
"""

In [43]:
def remove_blank_lines(input_string):
    # Split the string into lines
    lines = input_string.splitlines()
    # Filter out empty lines
    non_empty_lines = [line for line in lines if line.strip()]
    # Join the non-empty lines back into a single string
    output_string = '\n'.join(non_empty_lines)
    return output_string

In [36]:
def build_cls_prompt(data, ans_1, ans_2, res_1, res_2, prompt):
    header = format_header(data['table_text'][0])
    table = format_nested_list(data['table_text'][1:])
    statement = data['statement']
    title = data['table_caption']
    prompt = prompt.replace("[TITLE]", title)
    prompt = prompt.replace("[HEADER]", header)
    prompt = prompt.replace("[STAT]", statement)
    prompt = prompt.replace("[ANS1]", str(ans_1))
    prompt = prompt.replace("[ANS2]", str(ans_2))
    prompt = prompt.replace("[RES1]", remove_blank_lines(res_1))
    prompt = prompt.replace("[RES2]", remove_blank_lines(res_2))
    return prompt

In [211]:
prompt_cls_l = []
for i, data in enumerate(dataset_same):
  prompt_i = build_cls_prompt(data, ans_1[i], ans_refine[i], dater_inst[i], num_inst[i], prompt_cls)
  prompt_cls_l.append(prompt_i)

In [105]:
prompt_cls_l = []
for i, data in enumerate(dataset_same):
  prompt_i = build_cls_prompt(data, combine_flag[i], num_ans[i], combine_inst[i], num_inst[i], prompt_cls)
  prompt_cls_l.append(prompt_i)

In [33]:
prompt_verif = """
Below is a table header regarding [TITLE]:
Header: [HEADER]
You're tasked with determing whether the given statement is True:
[STAT]
You have 2 answers with their corresponding reasoning processes derived by two different models. Answer [A] was derived by the A model. Answer [B] was derived by B model.
For the answers, 1 means True and 0 means False
Reasoning of Answer [A] is: [RES1]
Answer [A] is: [ANS1]
Reasoning of Answer [B] is: [RES2]
Answer [B] is: [ANS2]
Your task is to determine whether this question can be correctly answered by these two models. True means yes and False means no.
Therefore, the final answer is:
"""

In [62]:
prompt_verf_l = []
for i, data in enumerate(dataset_same):
  prompt_i = build_cls_prompt(data, combine_flag[i], num_ans[i], combine_inst[i], num_inst[i], prompt_verif)
  prompt_verf_l.append(prompt_i)

In [61]:
combine_flag = []
for f in combine_ans:
    if f == 'True':
        combine_flag.append(1)
    else:
        combine_flag.append(0)

In [213]:
file_path = "prompt_cls_tabfact.txt"
def write_list_to_txt(lst, file_path):
    with open(file_path, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')
write_list_to_txt(prompt_cls_l, file_path)

In [10]:
file_path = "prompt_bin_tabfact.txt"
def write_list_to_txt(lst, file_path):
    with open(file_path, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n\n')
write_list_to_txt(prompt_verf_l, file_path)

In [11]:
def read_txt_to_list(file_path):
    with open(file_path, 'r') as file:
        # Read lines from the file and strip newline characters
        lines = [line.strip() for line in file]

    # Combine lines that are not blank into a single string
    combined_strings = []
    current_string = ''
    for line in lines:
        if line:
            current_string += line + '\n'  # Add a space to separate lines
        elif current_string:
            combined_strings.append(current_string.strip())
            current_string = ''

    # Add the last combined string if it's not empty
    if current_string:
        combined_strings.append(current_string.strip())

    return combined_strings

# file_path = "prompt_cls_tabfact.txt"

file_path = "prompt_bin_tabfact.txt"
prompt_verf = read_txt_to_list(file_path)

In [4]:
prompt_verf_l = []
for text in prompt_verf:
    tmp = text.replace("determine whether this question", "determine whether this statement")
    prompt_verf_l.append(tmp)

In [23]:
false_id = []
for i, x in enumerate(select_bin):
    if not x:
        false_id.append(i)

In [218]:
tab_lap_ans = []
for i, flag in enumerate(select_ans):
    if flag == "[A]":
        tab_lap_ans.append(num_solver_res[i])
    else:
        tab_lap_ans.append(dater_res[i])
len(tab_lap_ans)

2024

In [118]:
cnt = 0
for i, ans in enumerate(combine_res):
    if ans == ground_truth[i]:
        cnt += 1
cnt

1829

### Accuracy for TabLaP

In [114]:
1829 / 2024

0.9036561264822134

In [34]:
###binary
class UCBBandit:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.arm_counts = np.zeros(n_arms)  # number of arm chosen
        self.arm_rewards = np.zeros(n_arms)  # accumulate reward
        self.arm_means = np.zeros(n_arms)  # avg reward
        self.total_count = 0  # total number of choices

    def choose_arm(self):
        if self.total_count < self.n_arms:
            return self.total_count 
        else:
            ucb_values = self.arm_means + np.sqrt(2 * np.log(self.total_count) / self.arm_counts)
            return np.argmax(ucb_values)

    def update(self, chosen_arm, reward):
        self.total_count += 1
        self.arm_counts[chosen_arm] += 1
        self.arm_rewards[chosen_arm] += reward
        self.arm_means[chosen_arm] = self.arm_rewards[chosen_arm] / self.arm_counts[chosen_arm]

def mab_ucb(model_ans, sota_ans, combine_ans, test_labels, false_id):
    n_arms = 2
    n_rounds = len(false_id)
    bandit = UCBBandit(n_arms)

    round_ans = []
    for _ in range(5000):
        chosen_arms = []
        for i in range(n_rounds):
            fd = false_id[i]
            chosen_arm = bandit.choose_arm()
            if chosen_arm == 0:
                if combine_a[fd] != ground_truth[fd] and refine_a[fd] != ground_truth[fd]:
                    bandit.update(chosen_arm, 1)
                else:
                    bandit.update(chosen_arm, -1)
            else:
                if combine_res[fd] == ground_truth[fd]:
                    bandit.update(chosen_arm, 1)
                else:
                    bandit.update(chosen_arm, -1)
            chosen_arms.append(chosen_arm)
        round_ans.append(chosen_arms)

    cnt_all = []
    for chosen_arms in round_ans:
        cnt = 0
        for i in range(n_rounds):
            fd = false_id[i]
            if chosen_arms[i] == 0:
                if refine_a[fd] != ground_truth[fd] and combine_a[fd] != ground_truth[fd]:
                    cnt += 1
            else:
                if combine_res[fd] == ground_truth[fd]:
                    cnt += 1
        cnt_all.append(cnt)
    return np.mean(cnt_all)

In [35]:
correct_n = 0
for i in range(len(ground_truth)):
    if i not in false_id and ground_truth[i] == combine_a[i]:
        correct_n += 1
mab_num = mab_ucb(combine_a, refine_a, combine_res, ground_truth, false_id)
print("Tw Acc. of TabLaP with Multi-arm Bandits: ", (correct_n + mab_num) / len(ground_truth))

Tw Acc. of TabLaP with Multi-arm Bandits:  0.9001966403162056


In [219]:
i = 0
acc = 0.
for key in dic:
    to_union = collections.defaultdict(float)
    it = dic[key]
    table = it['data_item']['table_text']
    stat = it['data_item']['statement'].split('\n')[0]
    states_list.append(stat)

    pred = combine_ans[i]
    gt = it['data_item']['label']
    if gt == pred:
        acc += 1
    i += 1
print('ACC:', acc/len(dic))

ACC: 0.8848814229249012


### Tapex-large-finetuned-TabFact

In [31]:
from transformers import TapexTokenizer, BartForSequenceClassification
import pandas as pd

tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-tabfact")
model = BartForSequenceClassification.from_pretrained("microsoft/tapex-large-finetuned-tabfact")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'Refused', '1': 'Entailed'}. The number of labels wil be overwritten to 2.


In [32]:
data = {
    "year": [1896, 1900, 1904, 2004, 2008, 2012],
    "city": ["athens", "paris", "st. louis", "athens", "beijing", "london"]
}
table = pd.DataFrame.from_dict(data)

# tapex accepts uncased input since it is pre-trained on the uncased corpus
query = "beijing did not host the olympic games in 2012"
encoding = tokenizer(table=table, query=query, return_tensors="pt")

outputs = model(**encoding)
output_id = int(outputs.logits[0].argmax(dim=0))
print(model.config.id2label[output_id])

Entailed


In [33]:
table = df_list[0]
q = query[0]
encoding = tokenizer(table=table, query=q, return_tensors="pt")
outputs = model(**encoding)
output_id = int(outputs.logits[0].argmax(dim=0))
print(model.config.id2label[output_id])

Refused


In [24]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [41]:
tapex_res = []
for i in range(len(df_list)):
    table = df_list[i]
    q = queries[i]
    encoding = tokenizer(table=table, query=q, return_tensors="pt", truncation=True)
    outputs = model(**encoding)
    output_id = int(outputs.logits[0].argmax(dim=0))
    tapex_res.append(model.config.id2label[output_id])

In [50]:
cnt = 0
for i, flag in enumerate(tapex_res):
    tmp = 0
    if flag == 'Refused':
        tmp = 0
    else:
        tmp = 1
    if tmp == ground_truth[i]:
        cnt += 1
print("ACC: ", cnt/len(ground_truth))

ACC:  0.8522727272727273


### OmniTab

In [100]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large")
model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large")

In [53]:
data = {
    "year": [1896, 1900, 1904, 2004, 2008, 2012],
    "city": ["athens", "paris", "st. louis", "athens", "beijing", "london"]
}
table = pd.DataFrame.from_dict(data)

# tapex accepts uncased input since it is pre-trained on the uncased corpus
query = "beijing did not host the olympic games in 2012"
encoding = tokenizer(table=table, query=query, return_tensors="pt")

outputs = model.generate(**encoding)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

[' beijing']


In [4]:
prompt = "The table context is: " + "[TABLE]" + "\n"
prompt += "The statement is: " + "[QUESTION]" 
note = """
Final Answer: [ANSWER]
"""
prompt += note
prompt_omn = prompt
prompt_omn

'The table context is: [TABLE]\nThe statement is: [QUESTION]\nFinal Answer: [ANSWER]\n'

In [5]:
def generate_omn(table, question, ans, prompt_temp: str)->str:
  prompt = prompt_temp.replace("[ANSWER]", ans)
  prompt = prompt.replace("[TABLE]", table)
  prompt = prompt.replace("[QUESTION]", question)
  return prompt

In [1]:
### data load
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("wenhu/tab_fact", 'tab_fact')

# Access the train, validation, and test splits
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

/home/yuxiangw7/.local/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for wenhu/tab_fact contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wenhu/tab_fact
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [13]:
train_path = 'train.jsonl'

In [14]:
import json
dataset = []
with open(train_path) as f:
    lines = f.readlines()
    for line in lines:
        dic = json.loads(line)
        dataset.append(dic)

In [19]:
prompt = "The table header is " + "[HEADER]" + "\n"
prompt += "The table context is: " + "[TABLE]" + "\n"
prompt += "The statement is: " + "[STAT]" 
# note = """
# Final Answer: [ANSWER]
# """
# prompt += note
prompt_omn = prompt
prompt_omn

'The table header is [HEADER]\nThe table context is: [TABLE]\nThe statement is: [STAT]'

In [20]:
def generate_omni(table, stat, h, prompt_temp: str)->str:
#   prompt = prompt_temp.replace("[ANSWER]", ans)
  prompt = prompt_temp.replace("[TABLE]", table)
  prompt = prompt.replace("[STAT]", stat)
  prompt = prompt.replace("[HEADER]", h)
  return prompt

In [21]:
omnitab_train_prompt = []
for data in dataset:
    table = format_nested_list(data['table_text'][1:])
    header = format_header(data['table_text'][0])
    stat = data['statement']
    ans = str(data['label'])
    prompt_i = generate_omni(table, stat, header, prompt_omn)
    omnitab_train_prompt.append(prompt_i)

In [22]:
from datasets import Dataset, DatasetDict

dict_train = {"text": []}

for text in omnitab_train_prompt:
    dict_train["text"].append(text)

train_dataset = Dataset.from_dict(dict_train)
train_dataset = DatasetDict({"train": train_dataset})

print(len(train_dataset["train"]))  

92283


In [102]:
from datasets import Dataset, DatasetDict

dict_train = {"text": [], "answer": []}
questions = []
for i in range(10000):
    text = dataset[i]['table_text']
    ans = "Refused" if dataset[i]['label'] == 0 else "Entailed"
    dict_train["text"].append(text)
    dict_train["answer"].append(ans)
    questions.append(dataset[i]['statement'])
train_dataset = Dataset.from_dict(dict_train)

In [11]:
def preprocess_function(examples):
    inputs = examples["text"]
    tables = [
        pd.DataFrame.from_records(example_table[1:], columns=example_table[0])
        for example_table in inputs
    ]
    targets = examples["answer"]
    
    model_inputs = tokenizer(
        table=tables, query=questions, max_length=1024, padding="max_length", truncation=True
    )
    labels = tokenizer(
        answer=targets,
        max_length=10,
        padding="max_length",
        truncation=True,
    )
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [103]:
train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [104]:
dict_val = {"text": [], "answer": []}
questions = []
for i in range(10000,11000):
    text = dataset[i]['table_text']
    ans = "Refused" if dataset[i]['label'] == 0 else "Entailed"
    dict_val["text"].append(text)
    dict_val["answer"].append(ans)
    questions.append(dataset[i]['statement'])
val_dataset = Dataset.from_dict(dict_val)

In [105]:
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
dict_test = {"text": [], "answer": []}
questions = []
for i in range(len(dataset_same)):
    text = dataset_same[i]['table_text']
    ans = "Refused" if dataset_same[i]['label'] == 0 else "Entailed"
    dict_test["text"].append(text)
    dict_test["answer"].append(ans)
    questions.append(dataset_same[i]['statement'])
test_dataset = Dataset.from_dict(dict_test)

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "./omnitab_ft_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [16]:
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2024 [00:00<?, ? examples/s]

In [108]:
import transformers
from transformers import (
    AutoConfig,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)

In [109]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
)

In [132]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/data/gpfs/projects/punim2168/omnitab_output",
    overwrite_output_dir=False,
    do_train=True,
    do_eval=True,
    do_predict=True,
    num_train_epochs=2,              
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,   
    warmup_steps=1000,                
    weight_decay=0.01,               
    predict_with_generate=True,
    learning_rate=5e-06,
    optim='adamw_torch', 
    save_strategy='epoch', 
    evaluation_strategy='epoch',
    seed=42
)

/home/yuxiangw7/.local/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [133]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
)

In [134]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_130069/563698825.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [135]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.186500,0.902498
2,0.172200,0.865148


TrainOutput(global_step=5000, training_loss=0.17604901123046876, metrics={'train_runtime': 2047.0969, 'train_samples_per_second': 9.77, 'train_steps_per_second': 2.442, 'total_flos': 4.334209204224e+16, 'train_loss': 0.17604901123046876, 'epoch': 2.0})

In [136]:
trainer.save_model("./omnitab_ft_model")

In [137]:
if training_args.do_predict:
  print(("*** Test ***"))
  test_results = trainer.predict(
      test_dataset,
      metric_key_prefix="test",
      max_length=3,
      num_beams=None,
    )
  metrics = test_results.metrics
  max_test_samples = len(tokenized_datasets_test)
  metrics["test_samples"] = min(max_test_samples, len(tokenized_datasets_test))

*** Test ***


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [94]:
omni_res = []
tokenizer = trainer.tokenizer
for i in range(2024):
    res = tokenizer.decode([x for x in test_results[0][i] if x != -100])
    omni_res.append(res)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [18]:
model = model.to("cuda:0")

In [19]:
omnitab_res = []
for i in range(len(df_list)):
    table = df_list[i]
    q = queries[i]
    encoding = tokenizer(table=table, query=q, return_tensors="pt", truncation=True).to("cuda:0")
#     outputs = model(**encoding)
#     output_id = outputs.logits[0].argmax(dim=0)
#     omnitab_res.append(model.config.id2label[output_id])
    outputs = model.generate(**encoding)
    omnitab_res.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [20]:
cnt = 0
for i in range(len(test_dataset['answer'])):
    if test_dataset['answer'][i] == omnitab_res[i][0]:
        cnt += 1
#     else:
#         print(test_dataset['answer'][i], omnitab_res[i][0])
cnt

1524

In [21]:
1524 / 2024

0.7529644268774703

## GPT-3.5 Turbo

In [27]:
## Prompt template
prompt = "Please determine whether a given statement is true or not according to the given table regarding [TITLE] \n"
prompt += "The table header is: [HEADER]" + '\n'
prompt += "The table context is: " + "[TABLE]" + "\n"
prompt += "The statement is: " + "[STAT]" + "\n"
note = """
Notes:
- Try to solve the problem step by step and give the reasoning process
- Give me the answer in format "Final Answer: True/False" form (either True or False, without any explanation)
"""
prompt += note
prompt_direct = prompt
prompt_direct

'Please determine whether a given statement is true or not according to the given table regarding [TITLE] \nThe table header is: [HEADER]\nThe table context is: [TABLE]\nThe statement is: [STAT]\n\nNotes:\n- Try to solve the problem step by step and give the reasoning process\n- Give me the answer in format "Final Answer: True/False" form (either True or False, without any explanation)\n'

In [28]:
print(prompt_direct)

Please determine whether a given statement is true or not according to the given table regarding [TITLE] 
The table header is: [HEADER]
The table context is: [TABLE]
The statement is: [STAT]

Notes:
- Try to solve the problem step by step and give the reasoning process
- Give me the answer in format "Final Answer: True/False" form (either True or False, without any explanation)



In [31]:
## get prompt for each question
prompt_list = []
for data in dataset_same:
  prompt_i = generate_prompt(data, prompt_direct)
  prompt_list.append(prompt_i)

In [121]:
import requests

def request_gpt35(prompt, api_key, model="gpt-3.5-turbo-0125", temperature=0):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature
    }

    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

In [36]:
api_key = "YOUR_API_KEY"

In [4]:
def extract_reasoning_dp(input_string):
    final_answer_index = input_string.find('Final Answer')
    content = input_string[:final_answer_index].strip()
    return content

In [37]:
org_propose = []
instruction = []
refine_propose = []

for i in range(len(prompt_list)):
  print(i)
  filter_ans = ""
  response = request_gpt35(prompt_list[i], api_key)
  try:
      respon = response['choices'][0]['message']['content']
  except:
      org_propose.append('')
      instruction.append('')
      refine_propose.append('True')
      continue
  org_propose.append(respon)
  instruct = extract_reasoning_dp(respon)
  instruction.append(instruct)
  if "Final Answer" in respon:
      filter_ans = filter_answer(respon)
      refine_propose.append(filter_ans)
  else:
    refine_propose.append("no result")

0
['True']
1
['True']
2
['False']
3
['False']
4
['True']
5
['False']
6
['False']
7
['True']
8
['False']
9
['True']
10
['False']
11
['False']
12
['True']
13
['True']
14
['False']
15
['False']
16
['False']
17
['True']
18
['True']
19
['False']
20
['True']
21
['False']
22
['True']
23
['True']
24
['False']
25
['True']
26
['True']
27
['False']
28
['True']
29
['False']
30
['False']
31
['True']
32
['True']
33
['True']
34
['False']
35
['True']
36
['True']
37
['True']
38
['False']
39
['False']
40
['False']
41
['False']
42
['False']
43
['False']
44
['True']
45
['True']
46
['False']
47
['False']
48
['False']
49
['True']
50
['True']
51
['True']
52
['False']
53
['False']
54
['False']
55
['False']
56
['False']
57
['True']
58
['False']
59
['False']
60
['True']
61
['True']
62
['False']
63
['True']
64
['False']
65
['False']
66
['True']
67
['False']
68
['False']
69
['False']
70
['False']
71
['False']
72
['False']
73
['False']
74
['True']
75
['False']
76
['True']
77
['False']
78
['True']
79
['True']
80
['

['True']
613
['False']
614
['True']
615
['False']
616
['False']
617
['False']
618
['True']
619
['False']
620
['True']
621
['True']
622
['False']
623
['False']
624
['True']
625
['False']
626
['False']
627
['False']
628
['False']
629
['True']
630
['True']
631
['False']
632
['False']
633
['False']
634
['True']
635
['True']
636
['True']
637
['True']
638
['True']
639
['True']
640
['False']
641
['False']
642
['True']
643
['True']
644
['True']
645
['True']
646
['False']
647
['False']
648
['False']
649
['False']
650
['True']
651
['False']
652
['False']
653
['False']
654
['False']
655
['False']
656
['False']
657
['False']
658
['False']
659
['True']
660
['False']
661
['True']
662
['False']
663
['False']
664
['False']
665
['True']
666
['False']
667
['True']
668
['False']
669
['False']
670
['False']
671
['False']
672
['False']
673
['False']
674
['True']
675
['True']
676
['False']
677
['False']
678
['False']
679
['True']
680
['True']
681
['False']
682
['True']
683
['True']
684
['False']
685
['True'

['False']
1202
['True']
1203
['True']
1204
['False']
1205
['True']
1206
['True']
1207
['True']
1208
['True']
1209
['False']
1210
['True']
1211
['True']
1212
['True']
1213
['True']
1214
['True']
1215
['True']
1216
['True']
1217
['True']
1218
['False']
1219
['False']
1220
['False']
1221
['True']
1222
['False']
1223
['True']
1224
['False']
1225
['False']
1226
['False']
1227
['False']
1228
['True']
1229
['False']
1230
['False']
1231
['True']
1232
['True']
1233
['False']
1234
['False']
1235
['True']
1236
['True']
1237
['True']
1238
['True']
1239
['True']
1240
['True']
1241
['True']
1242
['False']
1243
['False']
1244
['False']
1245
['False']
1246
['True']
1247
['True']
1248
['False']
1249
['False']
1250
['False']
1251
['False']
1252
['False']
1253
['True']
1254
['False']
1255
['False']
1256
['True']
1257
['True']
1258
['False']
1259
['False']
1260
['False']
1261
['True']
1262
['False']
1263
['False']
1264
['False']
1265
['True']
1266
['False']
1267
['False']
1268
['False']
1269
['False']
127

['True']
1765
['True']
1766
['False']
1767
['True']
1768
['True']
1769
['False']
1770
['True']
1771
['True']
1772
['False']
1773
['True']
1774
['True']
1775
['False']
1776
['True']
1777
['True']
1778
['True']
1779
['True']
1780
['True']
1781
['True']
1782
['False']
1783
['False']
1784
['False']
1785
['False']
1786
['True']
1787
['False']
1788
['True']
1789
['False']
1790
['True']
1791
['True']
1792
['False']
1793
['False']
1794
['True']
1795
['False']
1796
['True']
1797
['True']
1798
['False']
1799
['True']
1800
['True']
1801
['True']
1802
['True']
1803
['False']
1804
['False']
1805
['True']
1806
['True']
1807
['False']
1808
['True']
1809
['False']
1810
['True']
1811
['True']
1812
['False']
1813
['True']
1814
['True']
1815
['True']
1816
['True']
1817
['False']
1818
['False']
1819
['True']
1820
['False']
1821
['True']
1822
['False']
1823
['False']
1824
['False']
1825
['False']
1826
['True']
1827
['True']
1828
['True']
1829
['False']
1830
['True']
1831
['True']
1832
['False']
1833
['Fals

In [50]:
refine_ans = []
for ans in refine_propose:
    refine_ans.append(ans[0])
refine_ans

['True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'True',
 'False',
 'True',
 'True',
 'False',
 'True',
 'True',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'False',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'Fal

In [54]:
cnt = 0
for i in range(len(refine_ans)):
    ans = 1
    if refine_ans[i] == "True":
        ans = 1
    else:
        ans = 0
    if ans == ground_truth[i]:
        cnt += 1
cnt

1670

In [55]:
1670 / 2024

0.825098814229249

## GPT4o-mini

In [37]:
org_4o = []
inst_4o = []
refine_4o = []

for i in range(len(prompt_list)):
  print(i)
  filter_ans = ""
  response = request_gpt4(prompt_list[i], api_key)
  try:
      respon = response['choices'][0]['message']['content']
  except:
      org_4o.append('')
      inst_4o.append('')
      refine_4o.append('True')
      continue
  org_4o.append(respon)
  instruct = extract_reasoning_dp(respon)
  inst_4o.append(instruct)
  if "Final Answer" in respon:
      filter_ans = filter_answer(respon)
      refine_4o.append(filter_ans)
  else:
      refine_4o.append("no result")

0
['True']
1
['False']
2
['True']
3
['False']
4
['False']
5
['False']
6
['False']
7
['True']
8
['False']
9
['True']
10
['True']
11
['False']
12
['True']
13
['True']
14
['False']
15
['False']
16
['False']
17
['True']
18
['False']
19
['False']
20
['True']
21
['True']
22
['False']
23
['False']
24
['False']
25
['True']
26
['True']
27
['False']
28
['True']
29
['False']
30
['False']
31
['True']
32
['True']
33
['True']
34
['False']
35
['True']
36
['True']
37
['False']
38
['False']
39
['False']
40
['False']
41
['False']
42
['False']
43
['False']
44
['True']
45
['True']
46
['False']
47
['False']
48
['False']
49
['False']
50
['True']
51
['True']
52
['False']
53
['False']
54
['False']
55
['False']
56
['False']
57
['True']
58
['True']
59
['False']
60
['True']
61
['True']
62
['False']
63
['True']
64
['False']
65
['False']
66
['True']
67
['False']
68
['True']
69
['False']
70
['False']
71
['False']
72
['False']
73
['False']
74
['True']
75
['False']
76
['False']
77
['False']
78
['True']
79
['True']
80

['False']
614
['True']
615
['False']
616
['False']
617
['False']
618
['False']
619
['False']
620
['True']
621
['True']
622
['False']
623
['False']
624
['True']
625
['True']
626
['False']
627
['False']
628
['False']
629
['True']
630
['True']
631
['False']
632
['False']
633
['False']
634
['True']
635
['True']
636
['True']
637
['True']
638
['True']
639
['True']
640
['False']
641
['False']
642
['True']
643
['True']
644
['False']
645
['True']
646
['True']
647
['False']
648
['False']
649
['False']
650
['True']
651
['False']
652
['True']
653
['False']
654
['False']
655
['False']
656
['False']
657
['False']
658
['False']
659
['True']
660
['False']
661
['True']
662
['False']
663
['False']
664
['False']
665
['False']
666
['False']
667
['True']
668
['False']
669
['False']
670
['False']
671
['True']
672
['False']
673
['False']
674
['False']
675
['True']
676
['False']
677
['True']
678
['False']
679
['True']
680
['True']
681
['False']
682
['False']
683
['False']
684
['False']
685
['False']
686
['Tru

['False']
1204
['True']
1205
['True']
1206
['True']
1207
['False']
1208
['False']
1209
['False']
1210
['True']
1211
['True']
1212
['True']
1213
['True']
1214
['True']
1215
['False']
1216
['True']
1217
['False']
1218
['False']
1219
['False']
1220
['False']
1221
['True']
1222
['True']
1223
['False']
1224
['False']
1225
['False']
1226
['False']
1227
['False']
1228
['True']
1229
['False']
1230
['True']
1231
['True']
1232
['True']
1233
['False']
1234
['False']
1235
['True']
1236
['True']
1237
['True']
1238
['True']
1239
['True']
1240
['False']
1241
['True']
1242
['True']
1243
['False']
1244
['False']
1245
['False']
1246
['True']
1247
['True']
1248
['False']
1249
['False']
1250
['False']
1251
['False']
1252
['False']
1253
['True']
1254
['False']
1255
['True']
1256
['True']
1257
['True']
1258
['True']
1259
['False']
1260
['False']
1261
['True']
1262
['False']
1263
['False']
1264
['False']
1265
['True']
1266
['False']
1267
['False']
1268
['False']
1269
['False']
1270
['False']
1271
['True']
12

['True']
1769
['True']
1770
['False']
1771
['True']
1772
['False']
1773
['True']
1774
['True']
1775
['False']
1776
['True']
1777
['False']
1778
['True']
1779
['True']
1780
['True']
1781
['True']
1782
['False']
1783
['False']
1784
['False']
1785
['False']
1786
['False']
1787
['True']
1788
['True']
1789
['False']
1790
['True']
1791
['True']
1792
['True']
1793
['False']
1794
['False']
1795
['False']
1796
['False']
1797
['True']
1798
['False']
1799
['True']
1800
['True']
1801
['False']
1802
['True']
1803
['False']
1804
['False']
1805
['True']
1806
['True']
1807
['True']
1808
['True']
1809
['False']
1810
['True']
1811
['True']
1812
['False']
1813
['False']
1814
['True']
1815
['True']
1816
['True']
1817
['False']
1818
['False']
1819
['True']
1820
['False']
1821
['True']
1822
['False']
1823
['False']
1824
['False']
1825
['False']
1826
['True']
1827
['True']
1828
['True']
1829
['False']
1830
['True']
1831
['True']
1832
['False']
1833
['False']
1834
['True']
1835
['True']
1836
['True']
1837
['F

In [9]:
refine_ans_4o = []
for ans in refine_4o:
    refine_ans_4o.append(ans[0])
refine_ans_4o

In [93]:
## Performance
1794 / 2024

0.8863636363636364